# Выбор локации для скважины

Компания «ГлавРосГосНефть» занимается добычей нефти. Мне, как работнику данной компании, поручено определить регион для бурения новых скважин.

В моем распоряжении данные о пробах нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. 

**Цель исследования**: построить модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Дополнительно проанализировать возможную прибыль и риски техникой Bootstrap.

Условия задачи:
- Для обучения модели подходит только линейная регрессия (остальные — недостаточно предсказуемые).
- При разведке региона исследуют 500 точек, из которых с помощью машинного обучения выбирают 200 лучших для разработки.
Бюджет на разработку скважин в регионе — 10 млрд рублей.
- При нынешних ценах один баррель сырья приносит 450 рублей дохода. Доход с каждой единицы продукта составляет 450 тыс. рублей, поскольку объём указан в тысячах баррелей.
- После оценки рисков нужно оставить лишь те регионы, в которых вероятность убытков меньше 2.5%. Среди них выбирают регион с наибольшей средней прибылью.

**Ход исследования**

Данные для исследования получим из файлов `geo_data_0.csv`, `geo_data_1.csv`, `geo_data_2.csv`. О качестве данных ничего не известно, поэтому перед обучением моделей понадобится их проверить. Найдем ошибки данных и оценим их влияние на исследование. На этапе подготовки попробуем исправить критичные из них.

Исследование пройдет в четыре этапа:
 1. Изучение и подготовка данных.
 2. Обучение и проверка модели.
 3. Подготовка к расчету прибыли.
 4. Расчет прибыли и рисков.
 
Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

## Загрузка и подготовка данных

Для начала импортируем необходимые в работе библиотеки.

In [1]:
import pandas as pd
import numpy as np
from scipy import stats as st
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

Прочитаем файлы `geo_data_0.csv`, `geo_data_1.csv`, `geo_data_2.csv` из каталога `datasets` и сохраним их в одноименных переменных.

In [2]:
geo_data_0 = pd.read_csv('/datasets/geo_data_0.csv')
geo_data_1 = pd.read_csv('/datasets/geo_data_1.csv')
geo_data_2 = pd.read_csv('/datasets/geo_data_2.csv')

Составим первое впечатление о данных:
1. Получим общую информацию методом `info()`.
2. Выведем на экран первые пять строк таблицы методом `head()`. 
3. Оценим разброс значений методом `describe()`.

In [3]:
for data in (geo_data_0, geo_data_1, geo_data_2):
    data.info()
    display(data.head())
    display(data.describe())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.500419,0.250143,2.502647,92.500000
std,0.871832,0.504433,3.248248,44.288691
min,-1.408605,-0.848218,-12.088328,0.000000
25%,-0.072580,-0.200881,0.287748,56.497507
50%,0.502360,0.250252,2.515969,91.849972
75%,1.073581,0.700646,4.715088,128.564089
max,2.362331,1.343769,16.003790,185.364347


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,1.141296,-4.796579,2.494541,68.825000
std,8.965932,5.119872,1.703572,45.944423
min,-31.609576,-26.358598,-0.018144,0.000000
25%,-6.298551,-8.267985,1.000021,26.953261
50%,1.153055,-4.813172,2.011479,57.085625
75%,8.621015,-1.332816,3.999904,107.813044
max,29.421755,18.734063,5.019721,137.945408


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.002023,-0.002081,2.495128,95.000000
std,1.732045,1.730417,3.473445,44.749921
min,-8.760004,-7.084020,-11.970335,0.000000
25%,-1.162288,-1.174820,0.130359,59.450441
50%,0.009424,-0.009482,2.484236,94.925613
75%,1.158535,1.163678,4.858794,130.595027
max,7.238262,7.844801,16.739402,190.029838


Итак, таблицы состоит из 5 столбцов:
* id — уникальный идентификатор скважины;
* f0 — первый признак точки;
* f1 — второй признак точки;
* f2 — третий признак точки;
* product — объём запасов в скважине (тыс. баррелей).

Целевым признаком наших таблиц является столбец `product`. Влияющими признаками в нашем случае будут столбцы `f0`, `f1`, `f2`. 

Столбец `id` исключим из обучения нашей модели, т.к. его значения являются уникальными для каждой скважины.

In [4]:
def target_and_features(data):
    target = data['product']
    features = data.drop(['id', 'product'], axis=1)
    return target, features

target_0, features_0 = target_and_features(geo_data_0)
target_1, features_1 = target_and_features(geo_data_1)
target_2, features_2 = target_and_features(geo_data_2)

Поделим исходный набор данных на обучающую и валидационную выборки в соотношении 3:1 с помощью метода `train_test_split()`

In [5]:
def train_and_valid(features, target):
    (features_train, features_valid, 
    target_train, target_valid) = train_test_split(features,
                                                   target,
                                                   test_size=0.25,
                                                   random_state=777)
    return features_train, features_valid, target_train, target_valid
    
(features_0_train, features_0_valid, 
target_0_train, target_0_valid) = train_and_valid(features_0, target_0)

(features_1_train, features_1_valid, 
target_1_train, target_1_valid) = train_and_valid(features_1, target_1)

(features_2_train, features_2_valid, 
target_2_train, target_2_valid) = train_and_valid(features_2, target_2)

Проверим размеры полученных выборок функцией `shape`.

In [6]:
print(features_0_train.shape, features_0_valid.shape, 
      features_1_train.shape, features_1_valid.shape,
      features_2_train.shape, features_2_valid.shape)

(75000, 3) (25000, 3) (75000, 3) (25000, 3) (75000, 3) (25000, 3)


В данных присутствуют количественные признаки `f0`, `f1`, `f2` с разными разбросами значений. Нормализуем их с помощью функции `StandardScaler()` библиотеки `sklearn`.

In [7]:
scaler = StandardScaler()

def normalize(features_train, features_valid):
    scaler.fit(features_train)
    features_train = scaler.transform(features_train)
    features_valid = scaler.transform(features_valid)
    return features_train, features_valid

features_0_train, features_0_valid = normalize(features_0_train, features_0_valid)
features_1_train, features_1_valid = normalize(features_1_train, features_1_valid)
features_2_train, features_2_valid = normalize(features_2_train, features_2_valid)

### Выводы

В нашем распоряжении таблицы с данными о месторождениях нефти в трех регионах. На этапе подготовки данных мы определили целевой и влияющие признаки для обучения модели, разделили данные на обучающие и валидационные выборки в соотношении 3 к 1. Нормализовав количественные признаки, мы сможем добиться лучшего качества моделей. 

Теперь можно переходить к обучению моделей. 

## Обучение и проверка модели

Согласно требованиям для решения задачи необходимо использовать модель линейной регрессии.

Обучим модель линейной регрессии на обучающей выборке каждого региона и сохраним предсказания модели в переменных `predicted_valid`. Затем определим средний предсказанный запас сырья для региона и значение квадратного корня из средней квадратичной ошибки (RMSE).

In [8]:
model = LinearRegression()

def prediction(features_train, target_train, features_valid, target_valid):
    model.fit(features_train, target_train)
    predicted_valid = pd.Series(model.predict(features_valid), index=target_valid.index)
    return predicted_valid

predicted_0_valid = prediction(features_0_train, target_0_train, 
                               features_0_valid, target_0_valid)
predicted_1_valid = prediction(features_1_train, target_1_train, 
                               features_1_valid, target_1_valid)
predicted_2_valid = prediction(features_2_train, target_2_train, 
                               features_2_valid, target_2_valid)

print(f'Средний запас предсказанного сырья в регионе 0: {predicted_0_valid.mean()}\n'
      f'RMSE модели в регионе 0: {mean_squared_error(target_0_valid, predicted_0_valid, squared=False)}\n'
      f'Средний запас предсказанного сырья в регионе 1: {predicted_1_valid.mean()}\n'
      f'RMSE модели в регионе 1: {mean_squared_error(target_1_valid, predicted_1_valid, squared=False)}\n'
      f'Средний запас предсказанного сырья в регионе 2: {predicted_2_valid.mean()}\n'
      f'RMSE модели в регионе 2: {mean_squared_error(target_2_valid, predicted_2_valid, squared=False)}')

Средний запас предсказанного сырья в регионе 0: 92.71675294828383
RMSE модели в регионе 0: 37.99219303094202
Средний запас предсказанного сырья в регионе 1: 69.75032521418687
RMSE модели в регионе 1: 0.8911506041453012
Средний запас предсказанного сырья в регионе 2: 94.97788403611848
RMSE модели в регионе 2: 40.23018694848326


### Выводы

Обучив на данных каждого региона модель линейной регрессии мы смогли добиться лучшего качества модели для региона 1 (RMSE = 0.89). Модели регрессии для регионов 0 и 2 менее точны, в среднем ошибаются на 38 и 40 пунктов соответственно. Наибольший средний запас сырья модель предсказала для региона 2: около 95 тыс. баррелей, наименьший для региона 1: чуть меньше 70 тыс. баррелей.

## Подготовка к расчёту прибыли

Согласно условиям задачи, при разведке региона исследуют 500 точек, из которых с помощью машинного обучения выбирают 200 лучших для разработки, бюджет на разработку скважин в регионе — 10 млрд рублей, доход с каждой единицы продукта составляет 450 тыс. рублей. 

Создадим для каждой величины переменную и запишем туда ее значение.

In [9]:
COUNT = 500
COUNT_ML = 200
COSTS = 10_000_000_000
INCOME = 450_000

Тогда достаточный объём сырья для безубыточной разработки новой скважины можно определить, как частное от деления величины бюджета на произведение доходности с единицы продукта на число скважен для разработки.

In [10]:
min_capacity = COSTS / (COUNT_ML * INCOME)
min_capacity

111.11111111111111

Объем сырья необходимый для безубыточной разработки скважины составляет 111.11 тысяч баррелей, что выше предсказанных средних значений сырья в регионах.

Напишем функцию, которая подсчитает и вернет прибыль. Она получает на вход:
* список ответов target — объем сырья скважины;
* список вероятностей probabilities — предсказанный моделью объем сырья скважины;
* количество скважин для разработки count;
* доход от единицы продукта income;
* расходы на разработку скважин costs. 


Функция должна выбрать скважины с наибольшим предсказанным объемом сырья и на основе реальных значений подсчитать возможную прибыль.

In [11]:
def profit(target, probabilities, count, income, costs):
    probs_sorted = probabilities.sort_values(ascending=False) 
    selected = target[probs_sorted.index][:count]
    return sum(selected * income) - costs

### Выводы

Объем сырья необходимый для безубыточной разработки скважины составляет примерно 111.11 тысяч баррелей. Для анализа прибыли от разработки новых скважин в регионе была реализована специальная функция, которая позволит определить лучший регион для бурения новых скважин.

## Расчёт прибыли и рисков 

Рассчитаем для каждого региона прибыль от разработки 200 скважин с максимально предсказанным моделью обучения объемом сырья.

In [12]:
print(f'Прибыль с региона 0: {profit(target_0_valid, predicted_0_valid, COUNT_ML, INCOME, COSTS) / 1_000_000_000:.2} млрд рублей\n'
      f'Прибыль с региона 1: {profit(target_1_valid, predicted_1_valid, COUNT_ML, INCOME, COSTS) / 1_000_000_000:.2} млрд рублей\n'
      f'Прибыль с региона 2: {profit(target_2_valid, predicted_2_valid, COUNT_ML, INCOME, COSTS) / 1_000_000_000:.2} млрд рублей\n')

Прибыль с региона 0: 3.4 млрд рублей
Прибыль с региона 1: 2.4 млрд рублей
Прибыль с региона 2: 2.3 млрд рублей



Максимальная прибыль от разработки 200 скважин с наибольшим прогнозируемым объемом нефти составляет 3.4 млрд рублей для региона 0. 

Воспользуемся техникой Bootstrap, чтобы определить распределение прибыли, а также риск убытков для этого региона.

In [13]:
state = np.random.RandomState(12345)
    
values = []
for i in range(1000):
    target_0_subsample = target_0_valid.sample(replace=True, random_state=state, n=COUNT)
    prob_0_subsample =  predicted_0_valid[target_0_subsample.index]
    values.append(profit(target_0_subsample, prob_0_subsample, COUNT_ML, INCOME, COSTS))
    
values = pd.Series(values)

mean = round(values.mean() / 1_000_000, 2)
lower = round(values.quantile(q=0.025) / 1_000_000, 2)
upper = round(values.quantile(q=0.975) / 1_000_000_000, 2)

print(f'Средняя прибыль: {mean} млн рублей')
print(f'95-% доверительный интервал: {lower} млн рублей - {upper} млрд рублей')
print(f'Риск убытков: {(values < 0).mean():.1%}')

Средняя прибыль: 489.85 млн рублей
95-% доверительный интервал: -51.51 млн рублей - 1.05 млрд рублей
Риск убытков: 3.7%


Согласно условиям задачи риск отрицательной прибыли для выбранного региона должен быть не более 2.5%, таким образом, регион 0 не подходит для разработки новых скважин.

Определим среднюю прибыль и риск убытков для региона 1.

In [14]:
values = []
for i in range(1000):
    target_1_subsample = target_1_valid.sample(replace=True, random_state=state, n=COUNT)
    prob_1_subsample =  predicted_1_valid[target_1_subsample.index]
    values.append(profit(target_1_subsample, prob_1_subsample, COUNT_ML, INCOME, COSTS))
    
values = pd.Series(values)

mean = round(values.mean() / 1_000_000, 2)
lower = round(values.quantile(q=0.025) / 1_000_000, 2)
upper = round(values.quantile(q=0.975) / 1_000_000_000, 2)

print(f'Средняя прибыль: {mean} млн рублей')
print(f'95-% доверительный интервал: {lower} млн рублей - {upper} млрд рублей')
print(f'Риск убытков: {(values < 0).mean():.1%}')

Средняя прибыль: 575.7 млн рублей
95-% доверительный интервал: 135.24 млн рублей - 1.03 млрд рублей
Риск убытков: 0.4%


Средняя прибыль для региона 1 составляет 576 млн рублей с 95-% доверительным интервалом от 135.24 млн до 1.03 млрд рублей. Риск убытков для данного региона равен 0.4%.

Перейдем к проверке последнего региона.

In [15]:
values = []
for i in range(1000):
    target_2_subsample = target_2_valid.sample(replace=True, random_state=state, n=COUNT)
    prob_2_subsample =  predicted_2_valid[target_2_subsample.index]
    values.append(profit(target_2_subsample, prob_2_subsample, COUNT_ML, INCOME, COSTS))
    
values = pd.Series(values)

mean = round(values.mean() / 1_000_000, 2)
lower = round(values.quantile(q=0.025) / 1_000_000, 2)
upper = round(values.quantile(q=0.975) / 1_000_000_000, 2)

print(f'Средняя прибыль: {mean} млн рублей')
print(f'95-% доверительный интервал: {lower} млн рублей - {upper} млрд рублей')
print(f'Риск убытков: {(values < 0).mean():.1%}')

Средняя прибыль: 373.97 млн рублей
95-% доверительный интервал: -195.96 млн рублей - 0.92 млрд рублей
Риск убытков: 10.1%


Риск убытков для региона 2 слишком высокий, чтобы рассматривать его в качестве региона для бурения новых скважин.

### Выводы

Применив технику Bootstrap с 1000 выборок, мы нашли распределение прибыли от разработки скважин для каждого региона. Риск отрицательной прибыли для регионов 0 и 2 слишком высок, чтобы рассматривать их для решения задачи исследования. Таким образом, регион 1 со средней прибылью 576 млн с 95-% доверительным интервалом от от 135.24 млн до 1.03 млрд рублей и риском убытков 0.4% является лучшим регионом для разработки месторождений.

## Общий вывод

На основе данных о пробах нефти в трёх регионах было проведено обучение моделей линейной регрессии. Наилучшее качество показала модель для региона 1 со значением квадратного корня из средней квадратичной ошибки, равным 0.89. Для региона 0 и 2 значение метрики RMSE составляет 38 и 40 соответственно. Наибольший средний объем сырья модель предсказала для региона 2 (95 тыс. баррелей), но для безубыточной разработки объем сырья для каждой скважины должен быть не менее 111.11 тысяч баррелей.

Реализовав функцию для расчета прибыли от скважин с максимально предсказанным объемом нефти и воспользовавшись техникой Bootstrap, мы смогли расчитать среднюю прибыль, 95-% доверительный интервал и риск убытков от разработки новых скважин для каждого региона. Максимальную среднюю прибыль (576 млн рублей) с доверительным интервалом от 135.24 млн до 1.03 млрд рублей млн рублей и минимальный риск убытков (0.4%) показал регион 1, который будет предложен в качестве региона для бурения новых скважин.